In [1]:
#from python_client.accounts import BaseAccount
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient
from anchorpy import Program, Provider, Wallet
from solana.keypair import Keypair
from base64 import b64decode
import base58
import yaml
from dataclasses import dataclass
import json
import pathlib
import borsh_construct as borsh #from borsh_construct import U8, String, CStruct
import asyncio
from solana.rpc.websocket_api import connect
from asyncstdlib import enumerate

In [2]:
#!pip install pyyaml
#!pip install asyncstdlib

In [3]:
animal = borsh.CStruct(
     "counter" / borsh.U32,
 )

In [4]:
async def main(program_address):
    async with AsyncClient("http://localhost:8899") as client:
        res = await client.is_connected()
        resp= await client.get_account_info(program_address)
        info = resp["result"]["value"]
        bytes_data = b64decode(info["data"][0])
        print (f'info {info}')
        print (f'animal - {animal.parse(bytes_data)}')

In [5]:
def get_greeted_pub_key(GREETER_SEED='hello'):
    # programId
    with open('/home/gabri/example-helloworld/dist/program/helloworld-keypair.json') as f:
        keypair = Keypair.from_secret_key(bytes(json.load(f)[:32]))

    programId = keypair.public_key

    # payer
    config = yaml.safe_load(open('/home/gabri/.config/solana/cli/config.yml'))
    with open(config['keypair_path']) as f:
        payer = Keypair.from_secret_key(bytes(json.load(f)[:32]))
    
    greetedPubkey = PublicKey.create_with_seed(payer.public_key,GREETER_SEED,programId)
    return greetedPubkey

In [6]:
greetedPubkey = get_greeted_pub_key()
greetedPubkey

2bt1NMJds5g4W1LwAeoVsMcUSa9qPD2PywAapBuLnpZw

In [7]:
await main(greetedPubkey)

info {'data': ['DgAAAA==', 'base64'], 'executable': False, 'lamports': 918720, 'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj', 'rentEpoch': 0}
animal - Container: 
    counter = 14


In [8]:
anotherGreetedPubKey = get_greeted_pub_key('hello_hello')
await main(anotherGreetedPubKey)

info {'data': ['BgAAAA==', 'base64'], 'executable': False, 'lamports': 918720, 'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj', 'rentEpoch': 0}
animal - Container: 
    counter = 6


In [9]:
# getting multiple accounts

In [10]:
client= AsyncClient("http://localhost:8899")

In [11]:
programId = 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj'
await client.get_program_accounts(PublicKey(programId))

{'jsonrpc': '2.0',
 'result': [{'account': {'data': 'Mkpwq',
    'executable': False,
    'lamports': 918720,
    'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj',
    'rentEpoch': 0},
   'pubkey': '2bt1NMJds5g4W1LwAeoVsMcUSa9qPD2PywAapBuLnpZw'},
  {'account': {'data': '9tvh5',
    'executable': False,
    'lamports': 918720,
    'owner': 'GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj',
    'rentEpoch': 0},
   'pubkey': '97j9kXnLJsGcz6Nxu96rN5S6AEqhrkmuYqsWH1nmW632'}],
 'id': 1}

In [12]:
# ToDo - Try accounts subscribe

In [13]:
async def subscribe_accounts_and_print():
    async with connect("ws://localhost:8900") as websocket:
        await websocket.logs_subscribe()
        first_resp = await websocket.recv()
        print (f'first_resp {first_resp}')
        subscription_id = first_resp.result
        print (f'subscription_id {subscription_id}')
        next_resp = await websocket.recv()
        print(f'next_resp {next_resp}')
        await websocket.logs_unsubscribe(subscription_id)

In [37]:
await subscribe_accounts_and_print()

first_resp Ok(result=28, id=5)
subscription_id 28
next_resp LogsNotification(subscription=28, result=LogItemAndContext(context=Context(slot=71145), value=LogItem(err=None, signature='5NZEg5N1cL1BmwQtqdKnBmqFs2HiwPVVJ35kxud1MXNQjsZ8ytA51Vge4B8WfnjkG1eM5vko6VrjqRh6iqVYPhPD', logs=['Program GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj invoke [1]', 'Program log: Hello World Rust program entrypoint', 'Program log: Greeted 14 time(s)! Account AccountInfo { key: 2bt1NMJds5g4W1LwAeoVsMcUSa9qPD2PywAapBuLnpZw owner: GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj is_signer: false is_writable: true executable: false rent_epoch: 0 lamports: 918720 data.len: 4  data: 0e000000 ... }', 'Program GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj consumed 27144 of 200000 compute units', 'Program GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj success'])))


In [51]:
async def subscribe_accounts_and_print_enum(account_pubkey, l):
    async with connect("ws://localhost:8900") as websocket:
        await websocket.account_subscribe(account_pubkey, encoding='base64')
        first_resp = await websocket.recv()
        print (f'first_resp {first_resp}')
        subscription_id = first_resp.result
        print (f'subscription_id {subscription_id}')
        async for idx, msg in enumerate(websocket):
            
            l.append(msg)
            counter, _base = msg.result.value.data
            bytes_data = b64decode(counter)
            print (f'counter - {animal.parse(bytes_data)}')
            
            print (f'msg {msg}')
            
            if idx == 1:
                break
        await websocket.account_unsubscribe(subscription_id)

In [52]:
l = []
await subscribe_accounts_and_print_enum(greetedPubkey, l)

first_resp Ok(result=57, id=11)
subscription_id 57
counter - Container: 
    counter = 23
msg AccountNotification(subscription=57, result=AccountInfoAndContext(context=Context(slot=74524), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('FwAAAA==', 'base64'), executable=False, rent_epoch=0)))
counter - Container: 
    counter = 24
msg AccountNotification(subscription=57, result=AccountInfoAndContext(context=Context(slot=74582), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('GAAAAA==', 'base64'), executable=False, rent_epoch=0)))


In [53]:
l

[AccountNotification(subscription=57, result=AccountInfoAndContext(context=Context(slot=74524), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('FwAAAA==', 'base64'), executable=False, rent_epoch=0))),
 AccountNotification(subscription=57, result=AccountInfoAndContext(context=Context(slot=74582), value=AccountInfo(lamports=918720, owner=GcUtT2NrSk8TT3zVM4tvGvTwDgo7gnA7esXo4RKUfzWj, data=('GAAAAA==', 'base64'), executable=False, rent_epoch=0)))]